# Working with Features using the Dseqrecord class

Ways to modify the file:
- add feature (e.g fragment of interest)
- extract feature (too, fragment of interest)
- generating SEGUID
- write

Please refer to pydna documentation for additional methods